# sleep_node_static

スリープノードを固定して経路群を構築

書き出したデータは./static/に保存される

In [ ]:
from itertools import combinations
import json
import os
from glob import glob
from pprint import pprint
from collections import defaultdict
import random
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
from graphillion import GraphSet

from proposal_packages.gridgraph import GridGraph

%matplotlib inline

In [ ]:
# 必要な関数

project_dir = "./static/"

def get_min_hop_num(terminal):
    """
    ある2点間のパスの最小ホップ数を求める
    重み付きグラフやメトリックテーブルを与えないでmin_iterを使用すると
    最小ホップ数のサブグラフからイテレートすることを利用する
    
    arguments:
    * terminal(tuple)
       端点を表す(s, t)形式のタプル
    """
    return len(next(GraphSet.paths(terminal[0], terminal[1]).min_iter()))

def get_min_hop_paths(paths, terminal):
    min_hop = get_min_hop_num(terminal)
    return paths.graph_size(min_hop)

def get_paths_removed_nodes(paths, nodes):
    for node in nodes:
        paths = paths.excluding(node)
    return paths
    
def flatten_paths(paths):
    """
    パス集合を平滑化して、リンクを要素とするリストを返す
    """
    return [edge for path in paths for edge in path]

def select_sleep_nodes(grid_graph, terminal, num):
    """
    ある確率でスリープ状態になるノードを選択する
    """
    nodes = set(grid_graph.nx_graph.nodes()) - set(terminal)
    return random.sample(nodes, k=num)

def create_minhop_path(paths, nodes):
    for node in nodes:
        paths = paths.including(node)
    return paths.choice()

def hamming(graphset, G, upper_limit):
    L = []
    lower_limit = 0
    for e in graphset:
        # 重み付きリンクとして定義されているとき，辺を表すタプルは(i,j,w)と表される．
        # そこからi,jのみ取り出すためe[:2]としている
        e = e[:2]
        if e in G:
            L.append((e[0], e[1], -1))
            lower_limit -= 1
        elif (e[1], e[0]) in G:
            L.append((e[1], e[0], -1))
            lower_limit -= 1
        else:
            L.append((e[0], e[1], 1))

    # print((L, (lower_limit, lower_limit + upper_limit)))
    # print("L=", L, "lower_limit=", lower_limit, "lower_limit+upper_limit=", lower_limit + upper_limit)
    return GraphSet.graphs(linear_constraints=[(L, (lower_limit, lower_limit + upper_limit))])

def get_similar_paths(edgelist, terminal, path, upper_limit):
    """
    arguments:
    * edgelist
    * terminal(tuple)
      (start, target)を表すタプル
    * path
    * upper_limit
    """
    if path is None:
        return
    similar_graphset = hamming(edgelist, path, upper_limit)
    similar_paths = GraphSet.paths(terminal[0], terminal[1], graphset=similar_graphset)
    return similar_paths

def draw_candidates5(candidates, pathset_nodes):
    """
    とりあえず5本描画する用の設定
    """
    plt.figure(figsize=(n+n*2+60, n+n*2+60))
    for i,  path in enumerate(candidates):
        plt.subplot(11, 10, 1+i)
        grid_graph.draw(subgraph=path, optional_colored_nodes=pathset_nodes)

def hamming_between_candidates(paths):
    res = defaultdict(list)
    for _path1, _path2 in combinations(paths, 2):
        duplicate_link_num = len(set(_path1) - set(_path2))
        res[tuple(_path1)].append(duplicate_link_num)
        res[tuple(_path2)].append(duplicate_link_num)
    
    total = 0
    for path, l in res.items():
        if sum(l) > total:
            total = sum(l)
            determinated = path
    return determinated, res

def _project_exists():
    """
    ランダム選択の結果のjsonファイルを
    保存するディレクトリが存在するか確認する
    """
    if not os.path.exists(project_dir):
        os.mkdir(project_dir)

def _create_sequencial_filename(candidates):
    """
    ファイル名が重複しないように
    ファイル名末尾に通し番号を付ける
    """
    keys = list(candidates.keys())
    vals = list(candidates.values())
    filename = "sleep{}_candidate{}*.json".format(len(keys[0]), len(vals[0]))
    sequencial_file_numbers = []
    for matched in glob(project_dir + filename):
        matched = os.path.splitext(os.path.basename(matched))[0]
        sequencial_file_numbers.append(int(matched.split("_")[2]))
    if sequencial_file_numbers:
        next_num = max(sequencial_file_numbers) + 1
    else:
        next_num = 1
    return "sleep{}_candidate{}_{}.json".format(len(keys[0]), len(vals[0]), next_num)

def dump_candidates_json(candidates):
    """
    candidatesをjsonファイルにして書き出す
    
    書き出すjsonファイルの命名規則について
    sleep{sleep node num}_candidate{candidate path num}_{sequencial}.json
    """
    _project_exists()
    filename = _create_sequencial_filename(candidates)
    with open(project_dir + filename, "w") as result_json:
        result_json.write(json.dumps({str(k): v for k, v in candidates.items()}))

def load_candidates_json(filename):
    """
    candidatesの内容を書き出したjsonファイルを読み込んで辞書形式に復元する
    """
    str2tuple = lambda s: tuple(map(int, s[1:-1].split(",")))
    candidates = {}
    with open(project_dir + filename, "r") as f:
        temp_candidates = json.load(f)
        candidates_values = []
        for nodes, paths in temp_candidates.items():
            candidates_values = [list(map(tuple, path)) for path in paths]
            candidates[str2tuple(nodes)] = candidates_values
    return candidates

格子グラフの作成

In [ ]:
n = 8
grid_graph = GridGraph(n, n)
plt.figure(figsize=(n, n))
grid_graph.draw()

In [ ]:
s, t = 1, (n + 1) * (n + 1)

In [ ]:
GraphSet.set_universe(grid_graph.nx_graph.edges())

In [ ]:
ptahs = GraphSet.paths(s, t).graph_size(n * 2)

スリープするノードを選択

In [ ]:
sleep_nodes = select_sleep_nodes(grid_graph, (s, t), 15)
sleep_nodes

In [ ]:
plt.figure(figsize=(n, n))
grid_graph.draw(optional_colored_nodes=sleep_nodes)

パスの最小ホップを求める

In [ ]:
min_hop_s_t = get_min_hop_num((s, t))

最小ホップ数のパス集合を求める

In [ ]:
min_hop_paths = GraphSet.paths(s, t).graph_size(min_hop_s_t)
len(min_hop_paths)

スリープしているノードを除外したパス集合を求める

In [ ]:
for sleep_node in sleep_nodes:
    min_hop_paths = min_hop_paths.excluding(sleep_node)
len(min_hop_paths)

経路群を5本構築する

In [ ]:
_candidates = []
for i, path in enumerate(min_hop_paths.rand_iter()):
    _candidates.append(path)
    if i == 4:
        break
candidates = {tuple(sleep_nodes): _candidates}

経路群をJSONファイルに書き出す

In [ ]:
dump_candidates_json(candidates)

JSONファイルの経路群を復元する

In [ ]:
loaded_candidates = load_candidates_json("sleep15_candidate5_1.json")

経路群を描画

In [ ]:
draw_candidates5(list(loaded_candidates.values())[0], sleep_nodes)

1つの図にまとめて描画

In [ ]:
# 5本の経路群をまとめて描画
colors = ["c", "g", "b", "m", "r", "y"]
plt.figure(figsize=(n, n))
grid_graph.draw_multi_subgraphs(subgraphs=list(loaded_candidates.values())[0], colors=colors)

決定経路を描画

In [ ]:
determinated, _ = hamming_between_candidates(list(loaded_candidates.values())[0])
plt.figure(figsize=(n, n))
grid_graph.draw(subgraph=determinated, optional_colored_nodes=sleep_nodes)

経路群の各経路がもつ類似の代替経路（最小迂回で再構成できる経路）の本数

In [ ]:
for path in list(loaded_candidates.values())[0]:
    print(path, len(get_similar_paths(GraphSet.universe(), (s, t), path, 4).graph_size(min_hop_s_t)))